In [21]:
#imports
import numpy as np
import pandas as pd
import nltk

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats

import prepare
import explore

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

import warnings
warnings.filterwarnings('ignore')

In [2]:
#read file locally
df = pd.read_json('data.json')

In [3]:
df = prepare.clean_df(df,
              ['readme_contents'],
              'lemmatize',
              extra_words=['\'', 'td', '&#9', 'a', 'br', '&#9;', 'use'])

In [4]:
#split data
train, val, test = prepare.split_data(df)

In [5]:
train.head()

,repo,language,readme_contents,readme_contents_clean,rm_length,sentiment
946,/petergoldstein/dalli,Ruby,Dalli [![Tests](https://github.com/petergoldst...,dalli testshttpsgithubcompetergoldsteindalliac...,2076,0.9878
876,/thoughtbot/gitsh,Ruby,"# <img src=""assets/gitsh-logo.svg"" alt=""gitsh""...",img srcassetsgitshlogosvg altgitsh width180\n\...,3371,0.9786
638,/satwikkansal/wtfpython,Python,"<p align=""center""><img src=""/images/logo.png"" ...",p aligncenterimg srcimageslogopng altp\nh1 ali...,79754,1.0000
308,/jhollingworth/bootstrap-wysihtml5,JavaScript,"**This repo is no longer maintained. ""bootstra...",repo longer maintained bootstrap3wysiwyghttpsg...,4584,0.9967
510,/JesusFreke/smali,Java,### About\n\nsmali/baksmali is an assembler/di...,about\n\nsmalibaksmali assemblerdisassembler d...,975,0.9571


In [6]:
#check the split
train.shape, val.shape,test.shape

((588, 6), (196, 6), (197, 6))

In [7]:
train.language.unique()

array(['Ruby', 'Python', 'JavaScript', 'Java', 'Other'], dtype=object)

In [8]:
#set up X and y
X_train = train.readme_contents_clean
X_val = val.readme_contents_clean
X_test = test.readme_contents_clean
y_train = train.language
y_val = val.language
y_test = test.language

Count vectorizer 

In [15]:
#create, fit object
cv= CountVectorizer()
X_train_bow = cv.fit_transform(X_train)
X_val_bow = cv.transform(X_val)

In [9]:
# decision tree
empty = []
for i in range(1,9):
    tree = DecisionTreeClassifier(max_depth = i)
    tree.fit(X_train_bow,y_train)
    train_acc = tree.score(X_train_bow,y_train)
    val_acc = tree.score(X_val_bow,y_val)
    dict = {"i":i,"train_acc":train_acc,"val_acc":val_acc}
    empty.append(dict)
tree_acc = pd.DataFrame(empty)
tree_acc

,i,train_acc,val_acc
0,1,0.403061,0.413265
1,2,0.562925,0.576531
2,3,0.697279,0.698980
3,4,0.721088,0.683673
4,5,0.780612,0.739796
5,6,0.811224,0.739796
6,7,0.836735,0.750000
7,8,0.862245,0.734694


In [10]:
#random forest
empty = []
for i in range(1,15):
    rf = RandomForestClassifier(min_samples_leaf= i)
    rf.fit(X_train_bow,y_train)
    train_acc = rf.score(X_train_bow,y_train)
    val_acc = rf.score(X_val_bow,y_val)
    dict = {"min_sample_leaf":i,"train_acc":train_acc,"val_acc":val_acc}
    empty.append(dict)
rf_acc = pd.DataFrame(empty)
rf_acc


,min_sample_leaf,train_acc,val_acc
0,1,0.998299,0.806122
1,2,0.940476,0.826531
2,3,0.892857,0.775510
3,4,0.848639,0.780612
4,5,0.833333,0.790816
5,6,0.809524,0.714286
6,7,0.799320,0.755102
7,8,0.787415,0.729592
8,9,0.761905,0.673469
9,10,0.763605,0.653061


In [11]:
#Naive bayes
empty = []
for i in range(1,15):
    nb = MultinomialNB(alpha = i)
    nb.fit(X_train_bow,y_train)
    train_acc = nb.score(X_train_bow,y_train)
    val_acc = nb.score(X_val_bow,y_val)
    dict = {"train_acc":train_acc,"val_acc":val_acc}
    empty.append(dict)
nb_acc = pd.DataFrame(empty)
nb_acc

,train_acc,val_acc
0,0.976190,0.734694
1,0.923469,0.688776
2,0.880952,0.642857
3,0.835034,0.632653
4,0.818027,0.617347
5,0.789116,0.602041
6,0.760204,0.576531
7,0.748299,0.571429
8,0.744898,0.556122
9,0.738095,0.540816


In [12]:
#logistic regression
lm = LogisticRegression()
lm.fit(X_train_bow,y_train)
train_acc = nb.score(X_train_bow,y_train)
val_acc = nb.score(X_val_bow,y_val)
print(train_acc)
print(val_acc)

0.70578231292517
0.5306122448979592


Using TF-IDF

In [16]:
tfidf = TfidfVectorizer()
X_train_bow = tfidf.fit_transform(X_train)
X_val_bow = tfidf.transform(X_val)

In [17]:
#decision tree
empty = []
for i in range(1,9):
    tree = DecisionTreeClassifier(max_depth = i)
    tree.fit(X_train_bow,y_train)
    train_acc = tree.score(X_train_bow,y_train)
    val_acc = tree.score(X_val_bow,y_val)
    dict = {"i":i,"train_acc":train_acc,"val_acc":val_acc}
    empty.append(dict)
tree_acc = pd.DataFrame(empty)
tree_acc

,i,train_acc,val_acc
0,1,0.396259,0.413265
1,2,0.559524,0.591837
2,3,0.687075,0.719388
3,4,0.709184,0.714286
4,5,0.761905,0.709184
5,6,0.777211,0.709184
6,7,0.807823,0.678571
7,8,0.838435,0.714286


In [18]:
#random forest
empty = []
for i in range(1,15):
    rf = RandomForestClassifier(min_samples_leaf= i)
    rf.fit(X_train_bow,y_train)
    train_acc = rf.score(X_train_bow,y_train)
    val_acc = rf.score(X_val_bow,y_val)
    dict = {"min_sample_leaf":i,"train_acc":train_acc,"val_acc":val_acc}
    empty.append(dict)
rf_acc = pd.DataFrame(empty)
rf_acc

,min_sample_leaf,train_acc,val_acc
0,1,0.998299,0.806122
1,2,0.962585,0.836735
2,3,0.921769,0.826531
3,4,0.891156,0.775510
4,5,0.858844,0.739796
5,6,0.841837,0.719388
6,7,0.836735,0.739796
7,8,0.829932,0.744898
8,9,0.809524,0.724490
9,10,0.802721,0.724490


In [19]:
#Naive bayes
empty = []
for i in range(1,15):
    nb = MultinomialNB(alpha = i)
    nb.fit(X_train_bow,y_train)
    train_acc = nb.score(X_train_bow,y_train)
    val_acc = nb.score(X_val_bow,y_val)
    dict = {"train_acc":train_acc,"val_acc":val_acc}
    empty.append(dict)
nb_acc = pd.DataFrame(empty)
nb_acc

,train_acc,val_acc
0,0.906463,0.816327
1,0.901361,0.811224
2,0.897959,0.806122
3,0.897959,0.811224
4,0.894558,0.816327
5,0.892857,0.826531
6,0.892857,0.821429
7,0.892857,0.821429
8,0.886054,0.821429
9,0.882653,0.811224


In [20]:
#logistic regression
lm = LogisticRegression()
lm.fit(X_train_bow,y_train)
train_acc = nb.score(X_train_bow,y_train)
val_acc = nb.score(X_val_bow,y_val)
print(train_acc)
print(val_acc)

0.8656462585034014
0.7959183673469388
